load in dataset

In [16]:
import pandas as pd
%pip install folium


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [77]:
df = pd.read_csv(r'C:\Users\jmcrawford\OneDrive - Deloitte (O365D)\QUB Analytathon\sample analytathon data.txt')
display(df.head())

,Farm_ID,Farm_Location_Lat,Farm_Location_Lon,Farm_Postcode,Herd_Size,Farm_Type,Farm_Size_Acres,Biosecurity_Score,Cattle_Movement_In_Last_Year,Proximity_To_Other_Farms_Km,...,Test_Result,Has_TB_History_Farm,Avg_Rainfall_3M_Prior_MM,Avg_Temperature_3M_Prior_C,Badger_Density_Score_Local,Proximity_To_Known_Badger_Sett_Km,Days_Since_Last_Test,Test_Season,Image_Path_Environment,Environmental_Risk_Label
0,FARM_001,50.876543,-3.456789,EX15 1AB,180,Dairy,300,2,50,1.2,...,Positive,False,250,8.5,High,0.5,365,Spring,farm_environment_images/FARM_001_env.jpg,High_Wildlife_Contact
1,FARM_002,51.234567,-5.123456,SA62 6CD,250,Beef,500,4,10,3.5,...,Negative,True,300,5.0,Low,5.0,180,Winter,farm_environment_images/FARM_002_env.jpg,Low_Wildlife_Contact
2,FARM_003,50.112233,-4.876543,TR11 2EF,120,Mixed,200,3,25,2.0,...,Positive,False,180,15.0,Medium,1.5,270,Autumn,farm_environment_images/FARM_003_env.jpg,Medium_Woodland_Proximity
3,FARM_004,50.567890,-3.987654,PL20 6GH,310,Dairy,400,2,70,0.8,...,Negative,True,220,10.0,High,0.7,90,Spring,farm_environment_images/FARM_004_env.jpg,High_Wildlife_Contact
4,FARM_005,51.098765,-2.765432,TA10 9IJ,200,Beef,350,4,15,4.0,...,Negative,False,280,7.0,Low,6.0,300,Autumn,farm_environment_images/FARM_005_env.jpg,Low_Wildlife_Contact


test data

In [78]:
# --- Data Generation Script (generate_analytathon_data.py) ---
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from faker import Faker
import random
from geopy.distance import geodesic # pip install geopy

fake = Faker('en_GB')

# --- Custom function to generate Northern Irish Postcodes (REFINED) ---
def generate_ni_postcode_refined():
    # A more limited and common set of NI postcode districts
    ni_districts = [
        'BT1', 'BT2', 'BT3', 'BT4', 'BT5', 'BT6', 'BT7', 'BT8', 'BT9', 'BT10',
        'BT11', 'BT12', 'BT13', 'BT14', 'BT15', 'BT16', 'BT17', 'BT18', 'BT19', 'BT20',
        'BT21', 'BT22', 'BT23', 'BT24', 'BT25', 'BT26', 'BT27', 'BT28', 'BT29', 'BT30',
        'BT31', 'BT32', 'BT33', 'BT34', 'BT35', 'BT36', 'BT37', 'BT38', 'BT39', 'BT40',
        'BT41', 'BT42', 'BT43', 'BT44', 'BT45', 'BT46', 'BT47', 'BT48', 'BT49', 'BT51',
        'BT52', 'BT53', 'BT54', 'BT55', 'BT56', 'BT57', 'BT60', 'BT61', 'BT62', 'BT63',
        'BT64', 'BT65', 'BT66', 'BT67', 'BT68', 'BT69', 'BT70', 'BT71', 'BT74', 'BT75',
        'BT76', 'BT77', 'BT78', 'BT79', 'BT80', 'BT81', 'BT82', 'BT92', 'BT93', 'BT94'
    ]
    
    prefix = random.choice(ni_districts)
    suffix_part1 = random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    suffix_part2 = random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    return f"{prefix} {random.randint(1,9)}{suffix_part1}{suffix_part2}"


def generate_analytathon_data(num_farms=1000, num_tests_per_farm=10, start_date='2020-01-01'):
    """Generates synthetic Bovine TB data with embedded trends, specific to Northern Ireland."""

    farm_ids = [f'FARM_{i:04d}' for i in range(num_farms)]
    
    # --- Geographic Bounding Box for Northern Ireland (WIDER TO INCLUDE WATER FOR CLEANING CHALLENGE) ---
    min_lat, max_lat = 54.05, 55.35
    min_lon, max_lon = -8.25, -5.35

    # --- Define SPECIFIC TB hotspots (Omagh and Downpatrick) ---
    # Each tuple: (latitude, longitude, radius_km)
    tb_hotspots = [
        (54.59, -7.30, 25), # Omagh hotspot (approx. center, 25km radius)
        (54.33, -5.70, 25)  # Downpatrick hotspot (approx. center, 25km radius)
    ]

    # --- Define a "Badger Sett" hotspot (can still exist as a separate risk factor) ---
    # Example: Fermanagh/Tyrone
    badger_sett_center = (54.8, -7.0) 
    
    farm_details = {}
    for farm_id in farm_ids:
        farm_location_lat = round(random.uniform(min_lat, max_lat), 6)
        farm_location_lon = round(random.uniform(min_lon, max_lon), 6)
        farm_postcode = generate_ni_postcode_refined()
        herd_size = random.randint(50, 500)
        farm_type = random.choice(['Dairy', 'Beef', 'Mixed'])
        farm_size_acres = random.randint(100, 1000)
        
        # Check if farm is in ANY of the defined TB hotspots
        is_in_tb_hotspot = False
        for h_lat, h_lon, h_radius in tb_hotspots:
            if geodesic((farm_location_lat, farm_location_lon), (h_lat, h_lon)).km < h_radius:
                is_in_tb_hotspot = True
                break
        
        # --- Generate risk factors: ONLY Proximity_To_Known_Badger_Sett_Km is biased by hotspot ---
        
        # Has_TB_History_Farm: Random for all farms
        has_tb_history = random.random() < 0.15 # 15% chance for any farm

        # Biosecurity_Score: Random for all farms
        biosecurity_score = random.randint(1, 5) # Normal range

        # Cattle_Movement_In_Last_Year: Random for all farms
        cattle_movement_in_last_year = random.randint(0, 150) # Normal range

        # Land_Use_Composition_Woodland_Pct: Random for all farms
        land_use_woodland_pct = random.randint(0, 30) # Normal range
            
        # Proximity_To_Known_Badger_Sett_Km: HEAVILY BIASED in hotspots
        if is_in_tb_hotspot:
            # Force closer proximity in TB hotspots
            proximity_to_badger_sett_km = round(random.uniform(0.1, 5.0), 1) # Very close
        else:
            # Normal range, potentially further away
            proximity_to_badger_sett_km = round(geodesic((farm_location_lat, farm_location_lon), badger_sett_center).km + random.uniform(-5, 10), 1)
            proximity_to_badger_sett_km = max(0.1, proximity_to_badger_sett_km) # Ensure non-negative
        
        # Determine Badger_Density_Score_Local based on proximity to sett hotspot (will naturally be 'High' in TB hotspots)
        if proximity_to_badger_sett_km < 10:
            badger_density_score = 'High'
        elif proximity_to_badger_sett_km < 30:
            badger_density_score = 'Medium'
        else:
            badger_density_score = 'Low'


        farm_details[farm_id] = {
            'Farm_Location_Lat': farm_location_lat,
            'Farm_Location_Lon': farm_location_lon,
            'Farm_Postcode': farm_postcode,
            'Herd_Size': herd_size,
            'Farm_Type': farm_type,
            'Farm_Size_Acres': farm_size_acres,
            'Biosecurity_Score': biosecurity_score,
            'Cattle_Movement_In_Last_Year': cattle_movement_in_last_year,
            'Proximity_To_Other_Farms_Km': round(random.uniform(0.5, 10.0), 1), # Not biased
            'Land_Use_Composition_Woodland_Pct': land_use_woodland_pct,
            'Has_TB_History_Farm': has_tb_history,
            'Is_In_TB_Hotspot': is_in_tb_hotspot,
            'Badger_Density_Score_Local': badger_density_score,
            'Proximity_To_Known_Badger_Sett_Km': proximity_to_badger_sett_km,
        }
    
    data = []
    for farm_id, details in farm_details.items():
        # Simulate a few tests per farm
        for i in range(num_tests_per_farm):
            test_date = pd.to_datetime(start_date) + timedelta(days=random.randint(0, 365 * 3)) # Over 3 years
            
            # --- Base probability of positive test (VERY LOW) ---
            prob_positive = 0.000005 

            # --- Increment probability based on embedded trends (Hotspot and Badger Proximity are key) ---
            if details['Has_TB_History_Farm']:
                prob_positive += 0.005 # Reduced influence
            if details['Is_In_TB_Hotspot']:
                prob_positive += 0.29 # KEPT HIGH to maintain ~30%
            
            # Environmental factors (woodland, badger proximity) still influence TB risk
            # Proximity to badger sett will be the dominant non-hotspot factor
            if details['Proximity_To_Known_Badger_Sett_Km'] < 2:
                prob_positive += 0.08 # Strong influence for very close proximity
            elif details['Proximity_To_Known_Badger_Sett_Km'] < 5:
                prob_positive += 0.03 # Moderate influence
            
            # Other factors have very small influence
            if details['Land_Use_Composition_Woodland_Pct'] > 20:
                prob_positive += 0.002
            if details['Biosecurity_Score'] < 3: # Low biosecurity
                prob_positive += 0.002
            if details['Cattle_Movement_In_Last_Year'] > 100: # High movement
                prob_positive += 0.001
            
            # Animal Age: very small influence
            animal_age_months = random.randint(6, 180)
            prob_positive += (animal_age_months / 180) * 0.0001 

            # Origin Type: very small influence
            origin_type = random.choice(['Born_On_Farm', 'Born_On_Farm', 'Purchased_From_Other_Farm'])
            if origin_type == 'Purchased_From_Other_Farm':
                prob_positive += 0.0005 

            # Test Season: very small influence
            test_season = pd.to_datetime(test_date).strftime('%B')
            if test_season in ['September', 'October', 'November', 'December']:
                prob_positive += 0.0002 
            
            # Add some random noise to the probability
            prob_positive = np.clip(prob_positive + np.random.normal(0, 0.0001), 0.000001, 0.35) 

            test_result = 'Positive' if random.random() < prob_positive else 'Negative'
            
            # Animal details
            animal_id = f'{farm_id}_ANIMAL_{random.randint(1, details["Herd_Size"]):03d}'
            sex = random.choice(['Male', 'Female'])
            breed = random.choice(['Holstein', 'Jersey', 'Angus', 'Hereford', 'Charolais', 'Limousin'])
            
            # Days since last test (simplified for this example)
            days_since_last_test = random.randint(90, 730) # 3 months to 2 years

            # Avg Rainfall/Temp (simplified, could be seasonal)
            avg_rainfall_3m_prior_mm = random.randint(100, 400)
            avg_temperature_3m_prior_c = random.uniform(4.0, 18.0)

            data.append({
                'Farm_ID': farm_id,
                'Farm_Location_Lat': details['Farm_Location_Lat'],
                'Farm_Location_Lon': details['Farm_Location_Lon'],
                'Farm_Postcode': details['Farm_Postcode'],
                'Herd_Size': details['Herd_Size'],
                'Farm_Type': details['Farm_Type'],
                'Farm_Size_Acres': details['Farm_Size_Acres'],
                'Biosecurity_Score': details['Biosecurity_Score'],
                'Cattle_Movement_In_Last_Year': details['Cattle_Movement_In_Last_Year'],
                'Proximity_To_Other_Farms_Km': details['Proximity_To_Other_Farms_Km'],
                'Land_Use_Composition_Woodland_Pct': details['Land_Use_Composition_Woodland_Pct'],
                'Animal_ID': animal_id,
                'Animal_Age_Months': animal_age_months,
                'Breed': breed,
                'Sex': sex,
                'Origin_Type': origin_type,
                'Test_Date': test_date,
                'Test_Result': test_result,
                'Has_TB_History_Farm': details['Has_TB_History_Farm'],
                'Is_In_TB_Hotspot': details['Is_In_TB_Hotspot'],
                'Avg_Rainfall_3M_Prior_MM': avg_rainfall_3m_prior_mm,
                'Avg_Temperature_3M_Prior_C': avg_temperature_3m_prior_c,
                'Badger_Density_Score_Local': details['Badger_Density_Score_Local'],
                'Proximity_To_Known_Badger_Sett_Km': details['Proximity_To_Known_Badger_Sett_Km'],
                'Days_Since_Last_Test': days_since_last_test,
                'Test_Season': test_season,
            })
            
    df = pd.DataFrame(data)
    
    # Introduce some missing values (e.g., 2-5% for some columns)
    for col in ['Animal_Age_Months', 'Breed', 'Sex', 'Origin_Type', 'Biosecurity_Score', 'Cattle_Movement_In_Last_Year']:
        missing_indices = np.random.choice(len(df), size=int(len(df) * random.uniform(0.02, 0.05)), replace=False)
        df.loc[missing_indices, col] = np.nan
    
    return df

# --- Run this part to generate and save the data ---
if __name__ == "__main__":
    df_analytathon = generate_analytathon_data(num_farms=1000, num_tests_per_farm=10)
    print(df_analytathon.head())
    print(f"\nTotal records: {len(df_analytathon)}")
    print(f"Positive tests: {df_analytathon['Test_Result'].value_counts().get('Positive', 0)}")
    print(f"Negative tests: {df_analytathon['Test_Result'].value_counts().get('Negative', 0)}")
    print(f"Missing values:\n{df_analytathon.isnull().sum()}")

    # Save to CSV for the challenge
    df_analytathon.to_csv('farm_data.csv', index=False)
    print("\nSynthetic Analytathon data saved to 'farm_data.csv'")

     Farm_ID  Farm_Location_Lat  Farm_Location_Lon Farm_Postcode  Herd_Size  \
0  FARM_0000            54.1664          -6.278712       BT1 2OV        309   
1  FARM_0000            54.1664          -6.278712       BT1 2OV        309   
2  FARM_0000            54.1664          -6.278712       BT1 2OV        309   
3  FARM_0000            54.1664          -6.278712       BT1 2OV        309   
4  FARM_0000            54.1664          -6.278712       BT1 2OV        309   

  Farm_Type  Farm_Size_Acres  Biosecurity_Score  Cattle_Movement_In_Last_Year  \
0      Beef              888                3.0                          75.0   
1      Beef              888                3.0                          75.0   
2      Beef              888                3.0                          75.0   
3      Beef              888                3.0                          75.0   
4      Beef              888                3.0                          75.0   

   Proximity_To_Other_Farms_Km  ...  T

validation

In [76]:
# --- Additional Validation for Embedded Hotspot Reasons (from previous response) ---

# Group by Is_In_TB_Hotspot and check means/distributions of other risk factors
print("\n--- Risk Factor Averages by Hotspot Status ---")

# Has_TB_History_Farm
print("\nMean Has_TB_History_Farm (True=1, False=0) by Hotspot Status:")
print(df_analytathon.groupby('Is_In_TB_Hotspot')['Has_TB_History_Farm'].mean()) # Should be similar

# Biosecurity_Score (lower is worse)
print("\nMean Biosecurity_Score by Hotspot Status (lower is worse):")
print(df_analytathon.groupby('Is_In_TB_Hotspot')['Biosecurity_Score'].mean()) # Should be similar

# Cattle_Movement_In_Last_Year
print("\nMean Cattle_Movement_In_Last_Year by Hotspot Status:")
print(df_analytathon.groupby('Is_In_TB_Hotspot')['Cattle_Movement_In_Last_Year'].mean()) # Should be similar

# Land_Use_Composition_Woodland_Pct
print("\nMean Land_Use_Composition_Woodland_Pct by Hotspot Status:")
print(df_analytathon.groupby('Is_In_TB_Hotspot')['Land_Use_Composition_Woodland_Pct'].mean()) # Should be similar

# Badger_Density_Score_Local (check distribution - should be heavily biased to 'High' in hotspots)
print("\nBadger_Density_Score_Local distribution by Hotspot Status:")
print(df_analytathon.groupby('Is_In_TB_Hotspot')['Badger_Density_Score_Local'].value_counts(normalize=True))

# Proximity_To_Known_Badger_Sett_Km (lower is closer - should be significantly lower in hotspots)
print("\nMean Proximity_To_Known_Badger_Sett_Km by Hotspot Status (lower is closer):")
print(df_analytathon.groupby('Is_In_TB_Hotspot')['Proximity_To_Known_Badger_Sett_Km'].mean())

# You could also plot these with seaborn for visual confirmation
# (e.g., boxplot for Proximity_To_Known_Badger_Sett_Km by Is_In_TB_Hotspot)


--- Risk Factor Averages by Hotspot Status ---

Mean Has_TB_History_Farm (True=1, False=0) by Hotspot Status:
Is_In_TB_Hotspot
False    0.043624
True     0.297030
Name: Has_TB_History_Farm, dtype: float64

Mean Biosecurity_Score by Hotspot Status (lower is worse):
Is_In_TB_Hotspot
False    3.360738
True     2.809406
Name: Biosecurity_Score, dtype: float64

Mean Cattle_Movement_In_Last_Year by Hotspot Status:
Is_In_TB_Hotspot
False    49.983221
True     90.116337
Name: Cattle_Movement_In_Last_Year, dtype: float64

Mean Land_Use_Composition_Woodland_Pct by Hotspot Status:
Is_In_TB_Hotspot
False    11.931208
True     22.121287
Name: Land_Use_Composition_Woodland_Pct, dtype: float64

Badger_Density_Score_Local distribution by Hotspot Status:
Is_In_TB_Hotspot  Badger_Density_Score_Local
False             Low                           0.651007
                  Medium                        0.348993
True              High                          0.653465
                  Medium           

In [74]:
def generate_ni_postcode_refined():
    """Generate realistic Northern Ireland postcodes."""
    prefixes = ['BT1', 'BT2', 'BT3', 'BT4', 'BT5', 'BT6', 'BT7', 'BT8', 'BT9', 'BT10', 
                'BT11', 'BT12', 'BT13', 'BT14', 'BT15', 'BT16', 'BT17', 'BT18', 'BT19', 'BT20',
                'BT21', 'BT22', 'BT23', 'BT24', 'BT25', 'BT26', 'BT27', 'BT28', 'BT29', 'BT30',
                'BT31', 'BT32', 'BT33', 'BT34', 'BT35', 'BT36', 'BT37', 'BT38', 'BT39', 'BT40',
                'BT41', 'BT42', 'BT43', 'BT44', 'BT45', 'BT46', 'BT47', 'BT48', 'BT49', 'BT50']
    prefix = random.choice(prefixes)
    number = random.randint(1, 9)
    letter_combo = ''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ', k=2))
    final_number = random.randint(1, 99)
    return f"{prefix} {number}{letter_combo} {final_number}"


def generate_farm_location_with_hotspot_bias(tb_hotspots, min_lat, max_lat, min_lon, max_lon, hotspot_probability=0.30):
    """Generate farm location with bias toward hotspots."""
    
    if random.random() < hotspot_probability:
        # Place farm IN a hotspot
        hotspot = random.choice(tb_hotspots)
        h_lat, h_lon, h_radius = hotspot
        
        # Generate point within circular hotspot
        angle = random.uniform(0, 2 * np.pi)
        distance_km = random.uniform(0, h_radius)
        
        # Convert to lat/lon (rough approximation)
        lat_offset = (distance_km / 111.0) * np.cos(angle)
        lon_offset = (distance_km / (111.0 * np.cos(np.radians(h_lat)))) * np.sin(angle)
        
        farm_lat = h_lat + lat_offset
        farm_lon = h_lon + lon_offset
        is_in_hotspot = True
    else:
        # Place farm OUTSIDE hotspots (but still in NI)
        farm_lat = round(random.uniform(min_lat, max_lat), 6)
        farm_lon = round(random.uniform(min_lon, max_lon), 6)
        
        # Double-check it's not accidentally in a hotspot
        is_in_hotspot = False
        for h_lat, h_lon, h_radius in tb_hotspots:
            if geodesic((farm_lat, farm_lon), (h_lat, h_lon)).km < h_radius:
                is_in_hotspot = True
                break
    
    return farm_lat, farm_lon, is_in_hotspot


def generate_analytathon_data_hotspot_fixed(num_farms=1000, num_tests_per_farm=10, start_date='2020-01-01'):
    """Generates synthetic Bovine TB data with CLEAR HOTSPOTS and realistic prevalence."""

    farm_ids = [f'FARM_{i:04d}' for i in range(num_farms)]
    
    min_lat, max_lat = 54.05, 55.35
    min_lon, max_lon = -8.25, -5.35

    tb_hotspots = [
        (54.59, -7.30, 25),  # Omagh hotspot (West)
        (54.33, -5.70, 25)   # Downpatrick hotspot (East)
    ]
    
    farm_details = {}
    
    for farm_id in farm_ids:
        # Generate location WITH hotspot bias
        farm_location_lat, farm_location_lon, is_in_tb_hotspot = generate_farm_location_with_hotspot_bias(
            tb_hotspots, min_lat, max_lat, min_lon, max_lon, hotspot_probability=0.30
        )
        
        farm_postcode = generate_ni_postcode_refined()
        
        # === DIFFERENTIATED FARM CHARACTERISTICS ===
        if is_in_tb_hotspot:
            proximity_to_badger_sett_km = round(random.uniform(0.5, 8.0), 1)
            badger_density_score = random.choice(['High', 'High', 'Medium'])
            biosecurity_score = random.choice([1, 1, 2, 3, 4, 5])
            herd_size = random.randint(100, 500)
            cattle_movement_in_last_year = random.randint(30, 150)
            farm_type = random.choice(['Dairy', 'Dairy', 'Mixed', 'Beef'])
            land_use_woodland_pct = random.randint(10, 35)
            has_tb_history = random.random() < 0.30
            proximity_to_other_farms_km = round(random.uniform(0.2, 5.0), 1)
        else:
            proximity_to_badger_sett_km = round(random.uniform(20.0, 120.0), 1)
            badger_density_score = random.choice(['Low', 'Low', 'Medium'])
            biosecurity_score = random.choice([2, 3, 3, 4, 5])
            herd_size = random.randint(50, 350)
            cattle_movement_in_last_year = random.randint(0, 100)
            farm_type = random.choice(['Dairy', 'Beef', 'Beef', 'Mixed'])
            land_use_woodland_pct = random.randint(0, 25)
            has_tb_history = random.random() < 0.05
            proximity_to_other_farms_km = round(random.uniform(0.5, 12.0), 1)
        
        farm_size_acres = random.randint(100, 1000)
        
        farm_details[farm_id] = {
            'Farm_Location_Lat': farm_location_lat,
            'Farm_Location_Lon': farm_location_lon,
            'Farm_Postcode': farm_postcode,
            'Herd_Size': herd_size,
            'Farm_Type': farm_type,
            'Farm_Size_Acres': farm_size_acres,
            'Biosecurity_Score': biosecurity_score,
            'Cattle_Movement_In_Last_Year': cattle_movement_in_last_year,
            'Proximity_To_Other_Farms_Km': proximity_to_other_farms_km,
            'Land_Use_Composition_Woodland_Pct': land_use_woodland_pct,
            'Has_TB_History_Farm': has_tb_history,
            'Is_In_TB_Hotspot': is_in_tb_hotspot,
            'Badger_Density_Score_Local': badger_density_score,
            'Proximity_To_Known_Badger_Sett_Km': proximity_to_badger_sett_km,
        }
    
    data = []
    
    for farm_id, details in farm_details.items():
        for i in range(num_tests_per_farm):
            test_date = pd.to_datetime(start_date) + timedelta(days=random.randint(0, 365 * 3))
            
            # === KEY FIX: DRAMATICALLY DIFFERENT BASELINES ===
            if details['Is_In_TB_Hotspot']:
                # HOTSPOTS: Start at 12% baseline (endemic region)
                prob_positive = 0.12
            else:
                # NON-HOTSPOTS: Start at 0.08% baseline (very rare)
                prob_positive = 0.0008
            
            # === ADDITIVE RISK FACTORS (REDUCED) ===
            
            # FACTOR 1: Badger proximity (reduced)
            if details['Proximity_To_Known_Badger_Sett_Km'] < 5:
                prob_positive += 0.03
            elif details['Proximity_To_Known_Badger_Sett_Km'] < 15:
                prob_positive += 0.01
            elif details['Proximity_To_Known_Badger_Sett_Km'] < 40:
                prob_positive += 0.002
            
            # FACTOR 2: Biosecurity score (reduced)
            if details['Biosecurity_Score'] == 1:
                prob_positive += 0.04
            elif details['Biosecurity_Score'] == 2:
                prob_positive += 0.02
            
            # FACTOR 3: Herd size (reduced)
            herd_size_normalized = min(details['Herd_Size'] / 500, 1.0)
            prob_positive += herd_size_normalized * 0.03
            
            # FACTOR 4: Cattle movement (reduced)
            movement_normalized = min(details['Cattle_Movement_In_Last_Year'] / 150, 1.0)
            prob_positive += movement_normalized * 0.02
            
            # FACTOR 5: Farm size (minimal)
            farm_size_normalized = min(details['Farm_Size_Acres'] / 1000, 1.0)
            prob_positive += farm_size_normalized * 0.01
            
            # FACTOR 6: Farm type (minimal)
            if details['Farm_Type'] == 'Dairy':
                prob_positive += 0.01
            
            # FACTOR 7: TB History (minimal)
            if details['Has_TB_History_Farm']:
                prob_positive += 0.015
            
            # FACTOR 8: Proximity to other farms (tiny)
            if details['Proximity_To_Other_Farms_Km'] < 2:
                prob_positive += 0.008
            elif details['Proximity_To_Other_Farms_Km'] < 5:
                prob_positive += 0.002
            
            # FACTOR 9: Seasonal variation (minimal)
            test_season = pd.to_datetime(test_date).strftime('%B')
            if test_season in ['September', 'October', 'November', 'December']:
                prob_positive += 0.002
            elif test_season in ['June', 'July', 'August']:
                prob_positive -= 0.001
            
            # FACTOR 10: Animal age (tiny)
            animal_age_months = random.randint(12, 180)
            if animal_age_months > 120:
                prob_positive += 0.001
            
            # FACTOR 11: Animal origin (tiny)
            origin_type = random.choice(['Born_On_Farm', 'Born_On_Farm', 'Purchased_From_Other_Farm'])
            if origin_type == 'Purchased_From_Other_Farm':
                prob_positive += 0.001
            
            # Add stochastic noise and clip to valid range
            prob_positive = np.clip(prob_positive + np.random.normal(0, 0.002), 0.00001, 0.60)
            
            test_result = 'Positive' if random.random() < prob_positive else 'Negative'
            
            animal_id = f'{farm_id}_ANIMAL_{random.randint(1, details["Herd_Size"]):03d}'
            sex = random.choice(['Male', 'Female'])
            breed = random.choice(['Holstein', 'Jersey', 'Angus', 'Hereford', 'Charolais', 'Limousin'])
            
            days_since_last_test = random.randint(90, 730)
            avg_rainfall_3m_prior_mm = random.randint(100, 400)
            avg_temperature_3m_prior_c = round(random.uniform(4.0, 18.0), 1)

            data.append({
                'Farm_ID': farm_id,
                'Farm_Location_Lat': details['Farm_Location_Lat'],
                'Farm_Location_Lon': details['Farm_Location_Lon'],
                'Farm_Postcode': details['Farm_Postcode'],
                'Herd_Size': details['Herd_Size'],
                'Farm_Type': details['Farm_Type'],
                'Farm_Size_Acres': details['Farm_Size_Acres'],
                'Biosecurity_Score': details['Biosecurity_Score'],
                'Cattle_Movement_In_Last_Year': details['Cattle_Movement_In_Last_Year'],
                'Proximity_To_Other_Farms_Km': details['Proximity_To_Other_Farms_Km'],
                'Land_Use_Composition_Woodland_Pct': details['Land_Use_Composition_Woodland_Pct'],
                'Animal_ID': animal_id,
                'Animal_Age_Months': animal_age_months,
                'Breed': breed,
                'Sex': sex,
                'Origin_Type': origin_type,
                'Test_Date': test_date,
                'Test_Result': test_result,
                'Has_TB_History_Farm': details['Has_TB_History_Farm'],
                'Is_In_TB_Hotspot': details['Is_In_TB_Hotspot'],
                'Avg_Rainfall_3M_Prior_MM': avg_rainfall_3m_prior_mm,
                'Avg_Temperature_3M_Prior_C': avg_temperature_3m_prior_c,
                'Badger_Density_Score_Local': details['Badger_Density_Score_Local'],
                'Proximity_To_Known_Badger_Sett_Km': details['Proximity_To_Known_Badger_Sett_Km'],
                'Days_Since_Last_Test': days_since_last_test,
                'Test_Season': test_season,
            })
    
    df = pd.DataFrame(data)
    return df


if __name__ == "__main__":
    df_analytathon = generate_analytathon_data_hotspot_fixed(num_farms=1000, num_tests_per_farm=10)
    
    print(f"Total records: {len(df_analytathon)}")
    print(f"Positive tests: {(df_analytathon['Test_Result'] == 'Positive').sum()}")
    print(f"Overall TB prevalence: {(df_analytathon['Test_Result'] == 'Positive').mean():.2%}")
    
    hotspot_df = df_analytathon[df_analytathon['Is_In_TB_Hotspot'] == True]
    non_hotspot_df = df_analytathon[df_analytathon['Is_In_TB_Hotspot'] == False]
    
    print(f"\n=== HOTSPOT ANALYSIS ===")
    print(f"Farms in hotspots: {hotspot_df['Farm_ID'].nunique()}")
    print(f"TB prevalence (hotspots): {(hotspot_df['Test_Result'] == 'Positive').mean():.1%}")
    print(f"Positive cases (hotspots): {(hotspot_df['Test_Result'] == 'Positive').sum()}")
    
    print(f"\nFarms outside hotspots: {non_hotspot_df['Farm_ID'].nunique()}")
    print(f"TB prevalence (non-hotspots): {(non_hotspot_df['Test_Result'] == 'Positive').mean():.3%}")
    print(f"Positive cases (non-hotspots): {(non_hotspot_df['Test_Result'] == 'Positive').sum()}")
    
    if (non_hotspot_df['Test_Result'] == 'Positive').mean() > 0:
        risk_ratio = (hotspot_df['Test_Result'] == 'Positive').mean() / (non_hotspot_df['Test_Result'] == 'Positive').mean()
        print(f"\n✓ Risk Ratio (Hotspot vs Non-Hotspot): {risk_ratio:.1f}x")
    
    df_analytathon.to_csv('farm_data.csv', index=False)
    print("\n✓ Data saved to 'farm_data.csv'")

Total records: 10000
Positive tests: 985
Overall TB prevalence: 9.85%

=== HOTSPOT ANALYSIS ===
Farms in hotspots: 404
TB prevalence (hotspots): 19.0%
Positive cases (hotspots): 769

Farms outside hotspots: 596
TB prevalence (non-hotspots): 3.624%
Positive cases (non-hotspots): 216

✓ Risk Ratio (Hotspot vs Non-Hotspot): 5.3x

✓ Data saved to 'farm_data.csv'
